# IMPORT PACKAGES AND DATA

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import initializers
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
data=np.load('/kaggle/input/data-assignment2/training_data.npy', allow_pickle=True)

In [ ]:
categories=np.load('/kaggle/input/data-assignment2/categories.npy', allow_pickle=True)

# EXPLORE DATA

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Get unique class labels and their counts
unique_classes, class_counts = np.unique(categories, return_counts=True)
print(class_counts)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(categories), y=categories)

# Create a dictionary mapping class indices to their respective weights
class_weights_dict = dict(zip(range(len(class_weights)), class_weights))

print("Class weights:", class_weights_dict)

In [ ]:
valid_periods=np.load('/kaggle/input/data-assignment2/valid_periods.npy', allow_pickle=True)

In [ ]:
recovered_series = []

for i in range(len(data)):
    start, end = valid_periods[i]
    series = data[i][start:end]
    recovered_series.append(series)

categorized_series = {category: [] for category in ['A', 'B', 'C', 'D', 'E', 'F']}

for i, category_code in enumerate(categories):
    category = category_code.item()  # Extract the string value
    categorized_series[category].append(recovered_series[i])

In [ ]:
quick_counter = 0
n_samples_cat = []
for category in ['A', 'B', 'C', 'D', 'E', 'F']:
    num_cat = len(categorized_series[category])
    quick_counter += num_cat
    print(f'There are {num_cat} time series of category {category}')
    n_samples_cat.append(num_cat)
    
categories = ['A', 'B', 'C', 'D', 'E', 'F']
num_time_series = [len(categorized_series[category]) for category in categories]

plt.figure(figsize=(8, 6))
plt.bar(categories, num_time_series, color='skyblue')
plt.xlabel('Categories')
plt.ylabel('Number of Time Series')
plt.title('Number of Time Series in Each Category')
plt.grid(axis='y')
plt.show()
    
print(quick_counter) # Check if everything's fine
print(n_samples_cat)

In [ ]:
# Access the first 10 elements of each category
first_10_elements = {category: categorized_series[category][:10] for category in categorized_series}

# Print the first 10 elements of each category along with their lengths
for category, elements in first_10_elements.items():
    print(f"Category {category}:")
    for idx, element in enumerate(elements, start=1):
        print(f"Element {idx}: Length - {len(element)}")
    print("\n")

In [ ]:
# Plot the first 10 elements of each category on different figures
for category, elements in categorized_series.items():
    plt.figure(figsize=(8, 6))
    plt.title(f"Category {category}")
    
    for idx, element in enumerate(elements[:10], start=1):
        plt.subplot(5, 2, idx)  # Create subplots for each element
        plt.plot(element)
        plt.title(f"Element {idx}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

all_train_data = []
all_test_data = []
all_val_data = []

for category_data in [categorized_series['A'], categorized_series['B'], categorized_series['C'], categorized_series['D'], categorized_series['E'], categorized_series['F']]:  # Add other categories as needed
    # Perform train-test-validation split for each category
    train_data, val_data = train_test_split(category_data, test_size=0.25, shuffle=True , random_state=2)
    #test_data, val_data = train_test_split(test_val_data, test_size=0.2, shuffle=True)  # Split the remaining for test and val
    
    # Append data from each category to the respective lists
    all_train_data.extend(train_data)
    # all_test_data.extend(test_data)
    all_val_data.extend(val_data)

In [ ]:
sequence_length = 200

np.random.shuffle(all_train_data)
np.random.shuffle(all_val_data)
#np.random.shuffle(all_test_data)

def create_sequences(data):
    input_sequences = []
    output_sequences = []
    for series in data:
        for i in range(len(series) - sequence_length - 9):  # Considering 9 samples as the prediction horizon
            input_sequences.append(series[i:i + sequence_length])
            output_sequences.append(series[i + sequence_length:i + sequence_length + 9])
    return np.array(input_sequences), np.array(output_sequences)

# Create input-output sequences for training, validation, and testing data
train_input, train_output = create_sequences(all_train_data)
val_input, val_output = create_sequences(all_val_data)
#test_input, test_output = create_sequences(all_test_data)

# Reshape data for RNN input (assuming 3D input shape [samples, time steps, features])
train_input = train_input.reshape(train_input.shape[0], sequence_length)
val_input = val_input.reshape(val_input.shape[0], sequence_length)
#test_input = test_input.reshape(test_input.shape[0], sequence_length)

In [ ]:
from tensorflow import keras

from keras import regularizers
from tensorflow.keras.layers import MultiHeadAttention, Dropout, LayerNormalization

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, GlobalAveragePooling1D, Concatenate, MaxPooling1D

import tensorflow as tf
import numpy as np

## RESNET WITH ATTENTION

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Dense, Flatten, Add, BatchNormalization, MultiHeadAttention, LayerNormalization

def build_model(input_shape=(200, 1), output_length=9):
    inputs = Input(shape=input_shape)

    # Convolutional layers
    conv1 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(inputs)
    conv1 = BatchNormalization()(conv1)

    conv2 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(conv1)
    conv2 = BatchNormalization()(conv2)

    # Self-attention layer
    attention_output = MultiHeadAttention(num_heads=8, key_dim=32)(conv2, conv2)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Residual connection from conv2 to attention_output
    residual_output = Add()([conv2, attention_output])

    conv3 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(residual_output)
    conv3 = BatchNormalization()(conv3)

    conv4 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(conv3)
    conv4 = BatchNormalization()(conv4)

    # Flatten and dense layers
    flatten = Flatten()(conv4)
    output = Dense(output_length, activation='linear')(flatten)  # Output for predicting 9 steps ahead

    # Create the model
    model = Model(inputs=inputs, outputs=output)

    return model

# Build the model
model = build_model()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')

# Display model summary
model.summary()


In [ ]:
model.fit(train_input, train_output, epochs=15, batch_size=128, validation_data=(val_input, val_output), callbacks=[tfk.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True), tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=1e-5)])